In [1]:
%run 10_kenlm_common.ipynb

/home/osx/anaconda3/envs/aac/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/osx/anaconda3/envs/aac/lib/python3.10/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/osx/anaconda3/envs/aac/lib/python3.10/site-packages/torch/cuda/__init__.py:740: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() if nvml_count < 0 else n

In [3]:
import os
input_dataset = "dataset_phonemes.txt"
output_directory = "output_directory/"
output_model_base = "output_model.klm"

# Create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Ngram size options
order_trigram = 3
order_unigram = 1

# Train

In [5]:
!kenlm/build/bin/lmplz -o 3    --text dataset_phonemes.txt     --arpa output_directory/output_model.klm_trigram.arpa     --discount_fallback --skip_symbols|     kenlm/build/bin/build_binary     -T /dev/stdin output_directory/output_model.klm_trigram.arpa

/home/osx/Documents/research/slang-ra/torgo_inference_on_cluster/JamSpell/build/kenlm/util/file.cc:76 in int util::OpenReadOrThrow(const char*) threw ErrnoException because `-1 == (ret = open(name, 00))'.
No such file or directory while opening output_directory/output_model.klm_trigram.arpa
ERROR
=== 1/5 Counting and sorting n-grams ===
Reading /home/osx/Documents/research/slang-ra/torgo_inference_on_cluster/dataset_phonemes.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 635344 types 42
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:504 2:10482238464 3:19654197248
Substituting fallback discounts for order 0: D1=0.5 D2=1 D3+=1.5
Statistics:
1 42 D1=0.5 D2=1 D3+=1.5
2 1280 D1=0.420074 D2=0.998284 D3+=1.12999
3 16435 D1=0.483725 D2=1.009 D3+=1.52672
Memory estimate for binary LM:
type     kB
probin

In [6]:
!kenlm/build/bin/build_binary\
    output_directory/output_model.klm_trigram.arpa \
    output_directory/output_model.klm_trigram.bin

Reading output_directory/output_model.klm_trigram.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
SUCCESS


# compress model file

In [7]:
!kenlm/build/bin/build_binary\
    output_directory/output_model.klm_trigram.arpa \
    output_directory/output_model.klm_trigram.klm

Reading output_directory/output_model.klm_trigram.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
SUCCESS


In [8]:
arpa_path = "output_directory/output_model.klm_trigram.arpa"
vocabulary = get_vocabulary_from_arpa(arpa_path)
# Print or use the vocabulary as needed
print("Vocabulary:", vocabulary)
# Tokens to add
new_tokens = ["<pad>", "<sil>", "<spn>"]
# Tokens to exclude
tokens_to_exclude = ["<s>", "</s>"]
# Add tokens to the ARPA file excluding the specified tokens
add_tokens_to_arpa(arpa_path, new_tokens, tokens_to_exclude)
vocabulary = get_vocabulary_from_arpa(arpa_path)

# Print or use the vocabulary as needed
print("Vocabulary:", vocabulary)

Vocabulary: {'3=16435', '1=42', '2=1280'}
Vocabulary: {'3=16435', '1=42', '2=1280'}


In [18]:
!ls

00_common.ipynb		  11_prepare_torgo_dataset.ipynb  output_directory
01_prepare_dataset.ipynb  12_prepare_kenlm_data.ipynb	  results
01_preprocess.ipynb	  13_g2p_kenlm_train.ipynb	  torgo.csv
02_bart_no_keep.ipynb	  data				  training_args.json
02_save_dataset.ipynb	  dataset_phonemes.txt		  train.py
03_bart_keep-all.ipynb	  JamSpell			  Untitled.ipynb
03_bart_keep.ipynb	  kenlm				  vocab
10_kenlm_common.ipynb	  logs


In [25]:
%cd JamSpell
%cd build

/home/osx/Documents/research/slang-ra/torgo_inference_on_cluster/JamSpell/build


# train

In [26]:
!./main/jamspell train ../test_data/alphabet_en.txt ../test_data/dataset_phonemes.txt model_torgo.bin


[info] loading text
[info] generating N-grams 1
[info] generating keys
[info] ngrams1: 39
[info] ngrams2: 1228
[info] ngrams3: 17135
[info] total: 18402
[info] generating perf hash
[info] finished, buckets: 23003
[info] buckets filled


In [33]:
%cd build
!ls

/home/osx/Documents/research/slang-ra/torgo_inference_on_cluster/JamSpell/build
CMakeCache.txt	cmake_install.cmake  jamspell  Makefile		tests
CMakeFiles	contrib		     main      model_torgo.bin	web_server


In [34]:
!python ../evaluate/evaluate.py\
	 -a ../test_data/alphabet_en.txt\
	 -jsp model_torgo.bin\
	 -mx 50000 ../test_data/dataset_phonemes.txt

Traceback (most recent call last):
  File "/home/osx/Documents/research/slang-ra/torgo_inference_on_cluster/JamSpell/build/../evaluate/evaluate.py", line 14, in <module>
    from evaluate import typo_model
  File "/home/osx/Documents/research/slang-ra/torgo_inference_on_cluster/JamSpell/evaluate/evaluate.py", line 14, in <module>
    from evaluate import typo_model
ImportError: cannot import name 'typo_model' from partially initialized module 'evaluate' (most likely due to a circular import) (/home/osx/Documents/research/slang-ra/torgo_inference_on_cluster/JamSpell/evaluate/evaluate.py)


In [42]:
import jamspell

corrector = jamspell.TSpellCorrector()
corrector.LoadLangModel('model_torgo.bin')

corrector.FixFragment('T AH D EY IH Z JH UW N AH N D IH T IH Z B ER TH D EQ')



'T AH D EY IH Z JH UW N AH N D IH T IH Z B ER TH D EY'